In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [3]:
import urllib.request
import urllib.parse
import json

#API받아오기 위한 정보 입력  # 발급받은 secret 입력 
client_id = 
client_secret = 

# 검색어와 출력 결과 갯수 입력받기
quote = input("검색어를 입력해주세요.: ")  # 검색어 입력받기
encText = urllib.parse.quote(quote)

total_results = int(input("총 가져올 검색 결과 갯수를 입력해주세요.: "))
display_num = 100  # 네이버 API는 한 번에 최대 100개까지만 출력할 수 있음

all_results = []

# 반복문을 통해 여러 번 요청 보내기
for start in range(1, total_results, display_num):
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display={display_num}&start={start}"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))
        all_results.extend(data['items'])
    else:
        print("Error Code:" + rescode)
        break

# 결과 출력
print(f"총 {len(all_results)}개의 결과를 가져왔습니다.")
for i, item in enumerate(all_results, start=1):
    print(f"{i}. {item['title']} - {item['link']}")


검색어를 입력해주세요.: "내돈내산""솔직""맛집""후기"
총 가져올 검색 결과 갯수를 입력해주세요.: 1000
총 1000개의 결과를 가져왔습니다.
1. [오사카 도톤보리 <b>맛집</b>] 츠케멘 스즈메 <b>내돈내산</b> 추천 <b>후기</b> - https://blog.naver.com/momentravel/223451876430
2. 무의도 카페 바다뷰가 멋있는 다츠도츠 <b>내돈내산 솔직</b><b>후기</b> - https://blog.naver.com/heeuee/223402613871
3. 투어 <b>내돈내산 솔직</b><b>후기</b>, 피렌체 여행, 이탈리아 투어 추천...  - https://blog.naver.com/fcoverbong/223453370752
4. [수원가정의달<b>맛집</b>]... 메뉴 넓은 매장 주차정보 <b>내돈내산</b>리뷰 - https://blog.naver.com/khing311/223437514153
5. 서울 시청역 <b>맛집</b> 퍼스트플로어 센트럴 <b>내돈내산</b> <b>맛집</b> - https://blog.naver.com/kakkong2/223444010278
6. 제주도 중문 <b>맛집</b> 제주미향 <b>내돈내산</b> 점심 <b>후기</b> - https://blog.naver.com/hyemin612/223439067216
7. [오사카 <b>맛집</b>] 난바역에서... 호라이만두&quot; <b>내돈내산 솔직</b><b>후기</b> - https://blog.naver.com/hyoinhwang/223448961053
8. 오이도 조개구이 <b>내돈내산</b> 청춘포차 푸짐했던 식사<b>후기</b>! - https://blog.naver.com/notgloom/223440440112
9. <b>맛집</b>* 흑돼지돈마호크 핵맛있음 탐라담 용인동백점 <b>내돈내산</b>...  - https://blog.naver.com/yahdd10044/223434618800
10. 청주 

In [4]:
# JSON 데이터를 문자열로 변환
body = json.dumps(all_results)

# 불필요한 ""(큰따옴표) 제거
body = body.replace('"','')

In [5]:
# body를 나누기
list1 = body.split('\n\t\t{\n\t\t\t')
# naver 블로그 글만 가져오기
list1 = [i for i in list1 if 'naver' in i]
list1

["[{title: [\\uc624\\uc0ac\\uce74 \\ub3c4\\ud1a4\\ubcf4\\ub9ac <b>\\ub9db\\uc9d1</b>] \\uce20\\ucf00\\uba58 \\uc2a4\\uc988\\uba54 <b>\\ub0b4\\ub3c8\\ub0b4\\uc0b0</b> \\ucd94\\ucc9c <b>\\ud6c4\\uae30</b>, link: https://blog.naver.com/momentravel/223451876430, description: 100% <b>\\ub0b4\\ub3c8\\ub0b4\\uc0b0 \\uc194\\uc9c1</b> <b>\\ud6c4 \\uae30</b> \\uc624\\uc0ac\\uce74 \\ub3c4\\ud1a4\\ubcf4\\ub9ac \\uce20\\ucf00\\uba58 <b>\\ub9db\\uc9d1</b> &quot;\\uc2a4\\uc988\\uba54&quot; <b>\\ub0b4\\ub3c8\\ub0b4\\uc0b0</b>\\uc73c\\ub85c \\ub2e4\\ub140\\uc654\\ub358 <b>\\ud6c4\\uae30</b>\\uc640 \\ubc14\\ub85c \\uc55e\\uc5d0 \\ubb35\\uc5c8\\ub358 \\ud638\\ud154 <b>\\ud6c4\\uae30</b>\\ub3c4 \\uc544\\ub798\\uc5d0 \\uae30\\uc7ac\\ud574\\ub460! \\ud6c4\\ucfe0\\uc624\\uce74\\uc5d0\\uc11c \\uba39\\uc5c8\\ub358 \\uce20\\ucf00\\uba58\\uc774 \\uc0dd\\uac01\\ubcf4\\ub2e4 \\ubcc4\\ub85c\\uc600\\uc5b4\\uc11c \\uae30\\ub300 \\uc548... , bloggername: \\ucc30\\ub098\\uc758 \\uc21c\\uac04, bloggerlink: blog.naver.co

In [6]:
titles = []
links = []

# 제목과 링크 추출
titles = [item['title'].replace("<b>", "").replace("</b>", "").strip() for item in all_results]
links = [item['link'].strip() for item in all_results]

# 결과 출력
print('<<제목 모음>>')
print(titles)
print('총 제목 수:', len(titles), '개')
print('\n<<링크 모음>>')
print(links)
print('총 링크 수:', len(links), '개')

<<제목 모음>>
['[오사카 도톤보리 맛집] 츠케멘 스즈메 내돈내산 추천 후기', '무의도 카페 바다뷰가 멋있는 다츠도츠 내돈내산 솔직후기', '투어 내돈내산 솔직후기, 피렌체 여행, 이탈리아 투어 추천...', '[수원가정의달맛집]... 메뉴 넓은 매장 주차정보 내돈내산리뷰', '서울 시청역 맛집 퍼스트플로어 센트럴 내돈내산 맛집', '제주도 중문 맛집 제주미향 내돈내산 점심 후기', '[오사카 맛집] 난바역에서... 호라이만두&quot; 내돈내산 솔직후기', '오이도 조개구이 내돈내산 청춘포차 푸짐했던 식사후기!', '맛집* 흑돼지돈마호크 핵맛있음 탐라담 용인동백점 내돈내산...', '청주 보리밥 맛집 대산보리밥 내돈내산 솔직후기', '성수 카페골목 가성비 술집 대림국수 주점 내돈내산 솔직 후기', '초코파이 / 안중근 장군 기념관 / 전주한옥마을 / 내돈내산 후기', '용인 맛집 한수위참치 내돈내산 솔직후기', '내돈내산 솔직후기 청주 가경동 중국집 향미각', '양꼬치 무한리필/병점 맛집 추천/내돈내산 더 많이 한집', '내돈내산 건대 마라탕 찐 맛집 솔직후기 : 백미원', '디즈니랜드 맛집] 호쿠사이 hokusai 일식당 내돈내산 후기!', '수원 대형카페 에스와이바자르 내돈내산 솔직후기', '신주쿠 테판 베이비, 또 가고 싶은 내돈내산 오꼬노미야끼 맛집', '[건대맛집] 가성비 소고기... 고기집 &quot;고기대통령&quot; 내돈내산 리뷰', '남천동맛집/안목/내돈내산돼지국밥맛집...', "해리단길 맛집] '솔솥'ㅣ한식 솥밥 추천, 내돈내산 솔직후기", '부천 신중동역 맛집 코이라멘 내돈내산 단골 후기', '용리단길 맛집에서 곱창전골부터 해장까지 내돈내산 솔직후기...', '하노이 Pho 10(포텐) - 솔직후기, 내돈내산, 맛집으로 소문난...', '백운호수 주차편한 삼동소바 , 카페 백운청 내돈내산 솔직후기', '*성남오마카세 추천* 레알 갓성비 스시미니 디너후기 내돈내산', '유럽 세미패키지 내돈내산 여기트래블 유럽퍼펙트 한달 후기', '샤

In [7]:
# 링크를 다듬기 (필요없는 부분 제거 및 수정)
blog_links = []
for i in links:
    a = i.replace('\\','')
    b = a.replace('?Redirect=Log&logNo=','/')
    # 다른 사이트 url이 남아있을 것을 대비해 한번 더 네이버만 남을 수 있게 걸러준다.
    if 'blog.naver.com' in b:
        blog_links.append(b)

print(blog_links)
print('생성된 링크 갯수:', len(blog_links), '개')

['https://blog.naver.com/momentravel/223451876430', 'https://blog.naver.com/heeuee/223402613871', 'https://blog.naver.com/fcoverbong/223453370752', 'https://blog.naver.com/khing311/223437514153', 'https://blog.naver.com/kakkong2/223444010278', 'https://blog.naver.com/hyemin612/223439067216', 'https://blog.naver.com/hyoinhwang/223448961053', 'https://blog.naver.com/notgloom/223440440112', 'https://blog.naver.com/yahdd10044/223434618800', 'https://blog.naver.com/jeombok0627/223400347569', 'https://blog.naver.com/karuni/223451738154', 'https://blog.naver.com/dlfdntjfql/223449676484', 'https://blog.naver.com/c_wonny/223444565237', 'https://blog.naver.com/gusdk233/223444031931', 'https://blog.naver.com/onaday1211/223442975274', 'https://blog.naver.com/thejlog/223432473120', 'https://blog.naver.com/marketing12/223407902371', 'https://blog.naver.com/ceb1248/223451755056', 'https://blog.naver.com/hatvo/223427413055', 'https://blog.naver.com/hyoinhwang/223440075260', 'https://blog.naver.com/ohm

In [8]:
# 본문 크롤링
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [9]:
contents = []
for i in blog_links:
    driver.get(i)
    time.sleep(1)
    driver.switch_to.frame("mainFrame")
    try:
        a = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text
        contents.append(a)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR, 'div#content-area').text
        contents.append(a)

driver.quit()
print("<<본문 크롤링이 완료되었습니다.>>")

<<본문 크롤링이 완료되었습니다.>>


In [10]:
# 제목, 블로그링크, 본문내용 Dataframe으로 만들기
df = pd.DataFrame({'제목': titles[:len(contents)], '링크': blog_links, '내용': contents})
print(df)

                                            제목   
0             [오사카 도톤보리 맛집] 츠케멘 스즈메 내돈내산 추천 후기  \
1               무의도 카페 바다뷰가 멋있는 다츠도츠 내돈내산 솔직후기   
2          투어 내돈내산 솔직후기, 피렌체 여행, 이탈리아 투어 추천...   
3           [수원가정의달맛집]... 메뉴 넓은 매장 주차정보 내돈내산리뷰   
4                 서울 시청역 맛집 퍼스트플로어 센트럴 내돈내산 맛집   
..                                         ...   
995                      다산 맛집 태능맥갈비 내돈내산 메뉴추천   
996  남양주 맛집 : 늘푸른정원... 식당, 최고였어요&quot; 내돈내산 후기   
997          여수 CGV 메가박스 근처 스시 맛집 :: 웅천초밥 내돈내산   
998       출연했던 인제 백담사 맛집 : 백담황태구이 (내돈내산 솔직 후기)   
999           찐으로 추천하는 숨어있는 맛집 추천/ 내돈내산 솔직후기❤️   

                                                  링크   
0    https://blog.naver.com/momentravel/223451876430  \
1         https://blog.naver.com/heeuee/223402613871   
2     https://blog.naver.com/fcoverbong/223453370752   
3       https://blog.naver.com/khing311/223437514153   
4       https://blog.naver.com/kakkong2/223444010278   
..                                               ...   
995    

In [11]:
# DataFrame을 CSV 파일로 저장
csv_file_name = "naver_blog_crawl_results_noAD.csv"
df.to_csv(csv_file_name, index=False, encoding='utf-8-sig')
print(f"데이터가 {csv_file_name} 파일로 저장되었습니다.")

데이터가 naver_blog_crawl_results_noAD.csv 파일로 저장되었습니다.
